In [1]:
input_string = "target area: x=34..67, y=-215..-186"
target_area = input_string.split(' ')
target_x_min, target_x_max = target_area[2].strip(',xy=').split('..')
target_y_min, target_y_max = target_area[3].strip(',xy=').split('..')
target_x_min, target_x_max, target_y_min, target_y_max = map(int, (target_x_min, target_x_max, target_y_min, target_y_max))


class Probe:
    def __init__(self, target_x_min, target_x_max, target_y_min, target_y_max):
        self.target_x_min = target_x_min
        self.target_x_max = target_x_max
        self.target_y_min = target_y_min
        self.target_y_max = target_y_max
        self.reset_pos()
        self.start_vel_x_equal_zero_at_target = set()
        self.steps_in_target_for_start_vel_x = dict()
        self.steps_in_target_for_start_vel_y = dict()
        self.validation_sets_y_max = dict()

    def reset_pos(self, vel_x=0, vel_y=0):
        self.pos_x = 0
        self.pos_y = 0
        self.vel_x = vel_x
        self.vel_y = vel_y
        self.pos_y_max = 0

    def iterate_vel_x(self):
        for start_vel_x in range(1, abs(self.target_x_max)+1):
            self.reset_pos(vel_x=start_vel_x)
            steps_in_target = set()
            for i_step in range(1000):
                self.step_x()
                if self.is_in_target_x():
                    steps_in_target.add(i_step)
                if (self.pos_x > target_x_max) or (self.vel_x == 0):
                    has_steps_in_target = len(steps_in_target) != 0
                    if has_steps_in_target:
                        self.steps_in_target_for_start_vel_x[start_vel_x] = steps_in_target
                        if self.vel_x == 0:
                            self.start_vel_x_equal_zero_at_target.add(start_vel_x)
                    break

    def iterate_vel_y(self):
        for start_vel_y in range(-abs(self.target_y_min), abs(self.target_y_min)+1):
            self.reset_pos(vel_y=start_vel_y)
            steps_in_target = set()
            for i_step in range(1000):
                self.step_y()
                if self.is_in_target_y():
                    steps_in_target.add(i_step)
                if self.pos_y < min(0, target_y_min):
                    has_steps_in_target = len(steps_in_target) != 0
                    if has_steps_in_target:
                        self.steps_in_target_for_start_vel_y[start_vel_y] = steps_in_target
                    self.validation_sets_y_max[start_vel_y] = self.pos_y_max
                    break

    def step_x(self):
        self.pos_x += self.vel_x
        if self.vel_x > 0:
            self.vel_x -= 1
        elif self.vel_x < 0:
            self.vel_x += 1

    def step_y(self):
        self.pos_y += self.vel_y
        self.vel_y -= 1
        self.set_y_max()

    def set_y_max(self):
        if self.pos_y > self.pos_y_max:
            self.pos_y_max = self.pos_y

    def is_in_target_x(self):
        return self.target_x_min <= self.pos_x <= self.target_x_max

    def is_in_target_y(self):
        return self.target_y_min <= self.pos_y <= self.target_y_max 

probe = Probe(target_x_min = target_x_min, target_x_max = target_x_max, target_y_min = target_y_min, target_y_max = target_y_max)
probe.iterate_vel_x()
probe.iterate_vel_y()

valid_y_heights = list()
for start_vel_x, steps_in_target_x in probe.steps_in_target_for_start_vel_x.items():
    for start_vel_y, steps_in_target_y in probe.steps_in_target_for_start_vel_y.items():
        if len(steps_in_target_x.intersection(steps_in_target_y)) > 0:
            valid_y_heights.append(probe.validation_sets_y_max[start_vel_y])
        elif (start_vel_x in probe.start_vel_x_equal_zero_at_target) and max(steps_in_target_x) <= max(steps_in_target_y):
            valid_y_heights.append(probe.validation_sets_y_max[start_vel_y])

print(max(valid_y_heights))
print(len(valid_y_heights))

23005
2040
